In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# テキストデータの処理
今まででてきた特徴量は，連続値とカテゴリデータである．  
現実ではこの2つに加えてテキストデータが出てくる．  
テキストデータの処理方法は，連続値ともカテゴリデータとも異なる．  

テキストデータは文字列で表される．  
しかし，文字列が全てテキストというわけではなく，カテゴリデータのこともある．  
文字列データは以下4つの種類に分類できる．  
- カテゴリデータ
- カテゴリに分類できるものの自由に書かれている文字列
- 構造化された文字列（名前や住所など）
- テキストデータ

カテゴリデータは4章で書いた通り処理する．  
カテゴリに分類できる自由な文字列は，最初は自動で処理し，後半は手動で分類していく．  
構造化された文字列は難しい（らしい）  
テキストデータは以下の通りに処理する．  
ただ，2024年現在ならDL使うのがいいんだろうけどね．

# Bag of Wordsによるテキスト表現
機械学習では，BoWが広く用いられている．  
BoWは単語の現れる回数を数えテキストを表現する方法である．  
BoW表現するには以下3ステップがある．  
1. トークン分割
2. ボキャブラリ構築
3. エンコード

これは，個々の文書を単語ごとに分割し，全ての文書で出てくる単語を数え，個々の文書に対して単語が現れる回数を数えるというステップ．

In [2]:
bards_words =["The fool doth think he is wise,",
              "but the wise man knows himself to be a fool"]

vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer()

In [11]:
# 出てきた単語数とその単語のID

print(len(vect.vocabulary_))
print(vect.vocabulary_)

13
{'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [12]:
# IDの単語がその文書で出てきた回数

bag_of_words = vect.transform(bards_words)
print(bag_of_words.toarray())

[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


# 実際の学習
実際の文書では，ある文書は全文書に出てきた単語の数の特徴量をもつことになる．  
この特徴量に対して以下3つの処理を行う（データによっては行わないほうがいいかも）
- 出現回数が少ない単語や，複数形や三単現などの変形した単語を消す
- 多すぎる単語(前置詞など)も意味がないので消す
- tf-idfなどを用いた重みづけ

そして，その特徴量に対してロジスティック回帰を用いて学習する．

### tf-idf
tf-idfは全文書にはあまり出てこないのに，ある文書には頻繁に現れる単語に大きな重みを与える．  
これは，他の文書にはでてこない単語はその文書の内容をよく表しているという発想．  
$N$を全文書の数，$N_w$を単語$w$が現れる文書の数とする．  
また，$tf$は文書$d$中に$w$が現れる回数である．  
すると，以下の式で$tfidf$は求められる．  

$tfidf(w, d) = tf(log(\frac{N_w+1}{N+1})+1)$

また，このtfidfはユークリッド長が1になるようにスケール変換をする．

# 1単語よりも大きい単位のBoW
1単語のBoWだと，単語の順番の情報が失われる．  
これはつまりnot goodなどの語彙が反映されないことを示している．  

この問題を解決するために，トークンをn-gramで特徴量とする方法が提案されている．  
単語（トークン）2つや3つをひとまとまりと考え，ひとまとまりを特徴量として使用する．  
ひとまとまりにする数がnのときn-gramとよぶ．

最小値から最大値を指定し，それらのn-gramでBoWを行う．  
最小値は1で，最大値は5くらいが精度向上に役にたつ．

In [6]:
bards_words =["The fool doth think he is wise,",
              "but the wise man knows himself to be a fool"]

cv = CountVectorizer(ngram_range=(1, 3)).fit(bards_words)
print("Vocabulary size: {}".format(len(cv.vocabulary_)))
print("Vocabulary:\n{}".format(cv.get_feature_names()))

Vocabulary size: 39
Vocabulary:
['be', 'be fool', 'but', 'but the', 'but the wise', 'doth', 'doth think', 'doth think he', 'fool', 'fool doth', 'fool doth think', 'he', 'he is', 'he is wise', 'himself', 'himself to', 'himself to be', 'is', 'is wise', 'knows', 'knows himself', 'knows himself to', 'man', 'man knows', 'man knows himself', 'the', 'the fool', 'the fool doth', 'the wise', 'the wise man', 'think', 'think he', 'think he is', 'to', 'to be', 'to be fool', 'wise', 'wise man', 'wise man knows']


### Pipelineで使う方法

In [13]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())

param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
              "tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)]}

# トークン1つ1つの処理
ひとつひとつのトークンの処理は，単数形・複数形や，現在形・過去形などを同一に処理したほうがいい．  
処理方法は，語幹を使って表現する方法やルールベースで処理する方法がある．  
これは性能がわずかによくなる程度だが，性能をぎりぎりまで良くしたい場合は有効である．

# 文章のトピックとクラスタリング
それぞれの文章が何をトピックとしているかを表す方法がある．  
それはクラスタリングだったり，成分分析だったりする．  
クラスタリングは文書をひとつのクラス(トピック)にわける方法で，成分分析は複数のトピックの組合せにわける．  

クラスタリングや成分分析では一般的な語彙は除いたほうがいいとされる．  
よって，頻出単語は削除することが多い．